In [3]:
import pandas
import numpy
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

In [4]:
sales = pandas.read_csv("home_data.csv")
sales.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,540088.141905,3.370842,2.114757,2079.899736,15106.967566,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,367127.195968,0.930062,0.770163,918.440897,41420.511515,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140825,685.391304,27304.179631
min,1.000102e+06,75000.000000,0.000000,0.000000,290.000000,520.000000,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155933,-122.518648,399.000000,651.000000
25%,2.123049e+09,321950.000000,3.000000,1.750000,1427.000000,5040.000000,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.470950,-122.328154,1490.000000,5100.000000
50%,3.904930e+09,450000.000000,3.000000,2.250000,1910.000000,7618.000000,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571794,-122.230495,1840.000000,7620.000000
75%,7.308900e+09,645000.000000,4.000000,2.500000,2550.000000,10688.000000,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.677985,-122.124764,2360.000000,10083.000000
max,9.900000e+09,7700000.000000,33.000000,8.000000,13540.000000,1651359.000000,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777624,-121.315091,6210.000000,871200.000000


# Question 1

In [5]:
meanPriceByZipCode = sales[['zipcode', 'price']].groupby('zipcode') \
    .aggregate(numpy.mean) \
    .sort_values(by='price',ascending=False)
meanPriceByZipCode[:1]

,price
zipcode,
98039,2160606


*Answer : 2160606*

# Question 2

In [6]:
calibratedSizeHouses = sales[(sales['sqft_living'] > 2000) & (sales['sqft_living'] <= 4000)]
calibratedSizeHouses['sqft_living'].describe()

count    9118.000000
mean     2670.773964
std       501.856351
min      2005.000000
25%      2250.000000
50%      2560.000000
75%      3010.000000
max      4000.000000
Name: sqft_living, dtype: float64

In [7]:
len(calibratedSizeHouses) / (1.0 * len(sales))

0.42187572294452413

*Answer : 0.42187572294452413*

# Question 3

In [25]:
basicFeatures = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']
advancedFeatures = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode','condition','grade','waterfront', 'view','sqft_above','sqft_basement','yr_built','yr_renovated', 'lat', 'long', 'sqft_living15','sqft_lot15']

# Question 3 : Scikit

In [34]:
from sklearn.cross_validation import train_test_split
from sklearn import linear_model

trainingSales,testSales = train_test_split(sales[features + ['price']], train_size=0.80, random_state=0)

skBasicModel = linear_model.LinearRegression()
skBasicModel.fit(trainingSales[basicFeatures], trainingSales[['price']])

skAdvancedModel = linear_model.LinearRegression()
skAdvancedModel.fit(trainingSales[advancedFeatures], trainingSales[['price']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [41]:
from sklearn.metrics import mean_squared_error
basicRMSE = mean_squared_error(testSales[['price']], skBasicModel.predict(testSales[basicFeatures]))**0.5
basicRMSE


244004.77443104264

In [43]:
advanceRMSE = mean_squared_error(testSales[['price']], skAdvancedModel.predict(testSales[advancedFeatures]))**0.5
advanceRMSE


190475.74644144482

In [44]:
basicRMSE - advanceRMSE

53529.027989597816

# Question 3 : SGraph

In [19]:
import graphlab
glSales = graphlab.SFrame('home_data.gl/')
train_data,test_data = glSales.random_split(.8,seed=0)

basicModel = graphlab.linear_regression.create(train_data, target='price', features=basicFeatures,validation_set=None)
advancedModel = graphlab.linear_regression.create(train_data, target='price', features=advancedFeatures,validation_set=None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 6
PROGRESS: Number of unpacked features : 6
PROGRESS: Number of coefficients    : 115
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.017012     | 3763208.270523     | 181908.848367 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PR

In [20]:
print basicModel.evaluate(test_data)

{'max_error': 3486584.509381705, 'rmse': 179542.4333126903}


In [21]:
print advancedModel.evaluate(test_data)

{'max_error': 3556849.413858208, 'rmse': 156831.1168021901}


In [24]:
basicModel.evaluate(test_data)['rmse'] - advancedModel.evaluate(test_data)['rmse']

22711.316510500183

*Answer : 22711.316510500183*